In [0]:
import numpy as np
from numpy import loadtxt
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [0]:
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
### Read data from txt format produced by EEGLAB to numpy array and optionally save numpy arrays to CSV in Google Drive ###
pathInMyDrive = "PsiloClassifier/txtDownsampledData/"
pathToSaveData = "PsiloClassifier/dataSplit/"

allSubjects = ["01", "02", "04", "05", "08", "10", "11", "12", "13", "14", "15", "16", "17", "20", "21", "22", "23", "24"]
testSubjects = ["02", "08", "15", "23"]
trainSubjects = [x for x in allSubjects if x not in testSubjects]

selectEveryNthChannelN = 5
sampleLength = 250
saveToDisk = False

trainX, trainY, testX, testY = load(selectEveryNthChannelN, sampleLength)

if(saveToDisk):
  np.savetxt(pathToSaveData + "trainX.csv", trainX, delimiter=",")
  np.savetxt(pathToSaveData + "trainY.csv", trainY, delimiter=",")
  np.savetxt(pathToSaveData + "testX.csv", testX, delimiter=",")
  np.savetxt(pathToSaveData + "testY.csv", testY, delimiter=",")

In [0]:
def load(selectEveryNthChannelN, sampleLength):
  trainXPlacebo, trainYPlacebo = loadProcess(trainSubjects, "PL", selectEveryNthChannelN, sampleLength)
  trainXHD, trainYHD = loadProcess(trainSubjects, "HD", selectEveryNthChannelN, sampleLength)
  trainX, trainY = catAndShuffleData(trainXPlacebo, trainYPlacebo, trainXHD, trainYHD)

  testXPlacebo, testYPlacebo = loadProcess(testSubjects, "PL", selectEveryNthChannelN, sampleLength)
  testXHD, testYHD = loadProcess(testSubjects, "HD", selectEveryNthChannelN, sampleLength)
  testX, testY = catAndShuffleData(testXPlacebo, testYPlacebo, testXHD, testYHD)

  return trainX, trainY, testX, testY;

In [0]:
def loadProcess(subjectList, dose, selectEveryNthChannelN, sampleLength):
  x = loadDownsampledDataFromSubjectListAndDose(subjectList, dose)
  x = keepEveryNthDataChannel(x, selectEveryNthChannelN)
  x = segmentChannelsIntoVectors(x, sampleLength)
  x = standardizeDataVectors(x)
  y = []
  if dose == "PL":
    for i in range(len(x)):
      y.append(np.zeros(1))
  else:
    for i in range(len(x)):
      y.append(np.ones(1))
  type(x)
  type(y)
  x = np.array(x)
  y = np.array(y)
  return x, y;

In [0]:
def loadDownsampledDataFromSubjectListAndDose(subjectList, dose):
  dataList = []
  for i in subjectList:
    filename = "S" + i + "_P300_" + dose + "_125.txt"
    dataList.append(loadtxt(pathInMyDrive + filename,  delimiter='\t'))
  return dataList

In [0]:
def keepEveryNthDataChannel(subjectDataList, n):
  dataList = []
  for subjectDataset in subjectDataList:
    dataList.append(subjectDataset[:, 0::n])
  return dataList

In [0]:
def segmentChannelsIntoVectors(subjectDataList, vectorLength):
  dataVectors = []
  for subjectDataset in subjectDataList:
    for channel in range(np.size(subjectDataset, 1)):
      time = int(np.random.uniform(0, vectorLength))
      while (time + vectorLength) < np.size(subjectDataset, 0):
        dataVectors.append(subjectDataset[time : time + vectorLength , channel])
        time = time + vectorLength                  
  return dataVectors

In [0]:
def standardizeDataVectors(dataVectorList):
  dataVectors = []
  for vec in dataVectorList:
    mean = np.mean(vec)
    stdev = np.std(vec)
    standardVec = (vec - mean)/stdev
    dataVectors.append(standardVec)
  return dataVectors

In [0]:
def catAndShuffleData(x0, y0, x1, y1):
  x = np.concatenate((x0, x1), axis=0)
  y = np.concatenate((y0, y1), axis=0)

  x, y = shuffle(x, y)

  return x, y;